In [11]:
import pandas as pd
import numpy as np
import glob

In [12]:
df = pd.read_csv('estat_hlth_cd_asdr2.csv')
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df.head()

,freq,unit,sex,age,icd10,geo,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,A,RT,F,TOTAL,A-R_V-Y,AT,817.74,844.98,826.55,795.74,820.93,779.04,797.24,784.67,765.68,819.09,807.4
1,A,RT,F,TOTAL,A-R_V-Y,AT1,857.95,897.23,864.51,835.1,852.43,823.36,836.93,836.28,799.23,845.58,859.88
2,A,RT,F,TOTAL,A-R_V-Y,AT11,843.83,912.63,865.16,862.3,848.15,826.53,823.99,814.31,800.48,856.48,830.61
3,A,RT,F,TOTAL,A-R_V-Y,AT12,860.17,909.24,857.47,826.01,863.52,825.72,839.69,831.56,810.46,835.14,871.85
4,A,RT,F,TOTAL,A-R_V-Y,AT13,860.27,884.4,872.69,838.31,841.58,820.75,835.68,842.79,784.87,851.54,851.16


In [13]:
path = '../ESTAT_CODELISTS/*.tsv'
files = glob.glob(path)
dataframes = {file.split('/')[-1].replace('.tsv', ''): pd.read_csv(
    file, sep='\t', on_bad_lines='skip') for file in files}

In [14]:
for col in ["freq", "unit", "sex", "age", "icd10", "geo"]:
    curr_codelist = dataframes[f"ESTAT_{col.upper()}"]
    df[col] = df[col].map(curr_codelist.set_index('CODE')['Label'])

df.rename(columns={'geo': 'country'}, inplace=True)
df.head()

,freq,unit,sex,age,icd10,country,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Annual,Rate,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Austria,817.74,844.98,826.55,795.74,820.93,779.04,797.24,784.67,765.68,819.09,807.4
1,Annual,Rate,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Ostösterreich,857.95,897.23,864.51,835.1,852.43,823.36,836.93,836.28,799.23,845.58,859.88
2,Annual,Rate,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Burgenland,843.83,912.63,865.16,862.3,848.15,826.53,823.99,814.31,800.48,856.48,830.61
3,Annual,Rate,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Niederösterreich,860.17,909.24,857.47,826.01,863.52,825.72,839.69,831.56,810.46,835.14,871.85
4,Annual,Rate,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Wien,860.27,884.4,872.69,838.31,841.58,820.75,835.68,842.79,784.87,851.54,851.16


In [15]:
df.replace(": z", np.nan, inplace=True)
df.replace(":", np.nan, inplace=True)

In [16]:
year_cols = [col for col in df.columns if col.isdigit()]

for col in year_cols:
    df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float)

In [17]:
df.head()

,freq,unit,sex,age,icd10,country,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Annual,Rate,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Austria,817.0,844.0,826.0,795.0,820.0,779.0,797.0,784.0,765.0,819.0,807.0
1,Annual,Rate,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Ostösterreich,857.0,897.0,864.0,835.0,852.0,823.0,836.0,836.0,799.0,845.0,859.0
2,Annual,Rate,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Burgenland,843.0,912.0,865.0,862.0,848.0,826.0,823.0,814.0,800.0,856.0,830.0
3,Annual,Rate,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Niederösterreich,860.0,909.0,857.0,826.0,863.0,825.0,839.0,831.0,810.0,835.0,871.0
4,Annual,Rate,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Wien,860.0,884.0,872.0,838.0,841.0,820.0,835.0,842.0,784.0,851.0,851.0


In [18]:
if df["freq"].nunique() == 1:
    df.drop(columns=["freq"], inplace=True)
if df["unit"].nunique() == 1:
    df.drop(columns=["unit"], inplace=True)

df.rename(columns={'icd10': 'causes'}, inplace=True)

In [19]:
df.head()

,sex,age,causes,country,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Austria,817.0,844.0,826.0,795.0,820.0,779.0,797.0,784.0,765.0,819.0,807.0
1,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Ostösterreich,857.0,897.0,864.0,835.0,852.0,823.0,836.0,836.0,799.0,845.0,859.0
2,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Burgenland,843.0,912.0,865.0,862.0,848.0,826.0,823.0,814.0,800.0,856.0,830.0
3,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Niederösterreich,860.0,909.0,857.0,826.0,863.0,825.0,839.0,831.0,810.0,835.0,871.0
4,Females,Total,All causes of death (A00-Y89) excluding S00-T98,Wien,860.0,884.0,872.0,838.0,841.0,820.0,835.0,842.0,784.0,851.0,851.0


In [20]:
df.to_csv("./clean/estat_hlth_cd_asdr2.csv", index=False)